In [1]:
import pandas as pd
import numpy as np

### resample()
resample()进行重采样。

重采样（Resampling）指的是把时间序列的频度变为另一个频度的过程。把高频度的数据变为低频度叫做降采样（downsampling），把低频度变为高频度叫做增采样（upsampling）。

### 降采样
考虑因素：

各区间哪边是闭合的（参数：closed）

如何标记各聚合面元，用区间的开头还是末尾（参数：label）

In [2]:
ts_index = pd.date_range('2018-08-03',periods =12,freq = 'T')
ts_index

DatetimeIndex(['2018-08-03 00:00:00', '2018-08-03 00:01:00',
               '2018-08-03 00:02:00', '2018-08-03 00:03:00',
               '2018-08-03 00:04:00', '2018-08-03 00:05:00',
               '2018-08-03 00:06:00', '2018-08-03 00:07:00',
               '2018-08-03 00:08:00', '2018-08-03 00:09:00',
               '2018-08-03 00:10:00', '2018-08-03 00:11:00'],
              dtype='datetime64[ns]', freq='T')

In [3]:
ts = pd.Series(np.arange(12),index = ts_index)
ts

2018-08-03 00:00:00     0
2018-08-03 00:01:00     1
2018-08-03 00:02:00     2
2018-08-03 00:03:00     3
2018-08-03 00:04:00     4
2018-08-03 00:05:00     5
2018-08-03 00:06:00     6
2018-08-03 00:07:00     7
2018-08-03 00:08:00     8
2018-08-03 00:09:00     9
2018-08-03 00:10:00    10
2018-08-03 00:11:00    11
Freq: T, dtype: int32

#### 默认使用左标签（label=‘left’），左闭合（closed='left’）

##### 此时第一个区间为：2018-08-03 00:00:00~2018-08-03 00:04:59，故sum为10，label为：2018-08-03 00:00:00

In [4]:
ts.resample('5min').sum()

2018-08-03 00:00:00    10
2018-08-03 00:05:00    35
2018-08-03 00:10:00    21
Freq: 5T, dtype: int32

#### 可以指定为右闭合（closed='right’），默认使用左标签（label=‘left’）

##### 此时第一个区间为：2018-08-02 23:55:01~2018-08-03 00:00:00，故sum为0，label为：2018-08-03 00:00:00

In [5]:
ts.resample('5min',closed='right',label='right').sum()

2018-08-03 00:00:00     0
2018-08-03 00:05:00    15
2018-08-03 00:10:00    40
2018-08-03 00:15:00    11
Freq: 5T, dtype: int32

## 升采样
考虑因素：

没有聚合，但是需要填充

In [6]:
frame = pd.DataFrame(np.random.randn(2, 4),index=pd.date_range('1/1/2000', periods=2,
                                                               freq='W-WED'),  # freq='W-WED'表示按周
                     columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame

,Colorado,Texas,New York,Ohio
2000-01-05,-0.38653,0.007634,0.199873,-0.525495
2000-01-12,-0.88979,0.940215,-0.539913,1.296344


#### 当我们对这个数据进行聚合的的时候，每个组只有一个值，以及gap（间隔）之间的缺失值。在不使用任何聚合函数的情况下，我们使用asfreq方法将其转换为高频度：

In [7]:
df_daily = frame.resample('D').asfreq()
df_daily

,Colorado,Texas,New York,Ohio
2000-01-05,-0.38653,0.007634,0.199873,-0.525495
2000-01-06,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,-0.88979,0.940215,-0.539913,1.296344


### 使用ffill()进行填充

In [8]:
frame.resample('D').ffill()

,Colorado,Texas,New York,Ohio
2000-01-05,-0.38653,0.007634,0.199873,-0.525495
2000-01-06,-0.38653,0.007634,0.199873,-0.525495
2000-01-07,-0.38653,0.007634,0.199873,-0.525495
2000-01-08,-0.38653,0.007634,0.199873,-0.525495
2000-01-09,-0.38653,0.007634,0.199873,-0.525495
2000-01-10,-0.38653,0.007634,0.199873,-0.525495
2000-01-11,-0.38653,0.007634,0.199873,-0.525495
2000-01-12,-0.88979,0.940215,-0.539913,1.296344


In [9]:
frame.resample('D').ffill(limit=2)

,Colorado,Texas,New York,Ohio
2000-01-05,-0.38653,0.007634,0.199873,-0.525495
2000-01-06,-0.38653,0.007634,0.199873,-0.525495
2000-01-07,-0.38653,0.007634,0.199873,-0.525495
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,-0.88979,0.940215,-0.539913,1.296344


### 新的日期索引没必要跟旧的重叠

In [10]:
frame.resample('W-THU').ffill()

,Colorado,Texas,New York,Ohio
2000-01-06,-0.38653,0.007634,0.199873,-0.525495
2000-01-13,-0.88979,0.940215,-0.539913,1.296344


## 分组重采样

In [11]:
times = pd.date_range('2018-08-3 00:00', freq='1min', periods=10)

In [13]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], 10),
                    'value': np.arange(30)})
df2[0:5]

,time,key,value
0,2018-08-03 00:00:00,a,0
1,2018-08-03 00:00:00,b,1
2,2018-08-03 00:00:00,c,2
3,2018-08-03 00:01:00,a,3
4,2018-08-03 00:01:00,b,4


In [14]:
df2.groupby(['key',pd.Grouper(key='time',freq='5min')]).sum()

value
key time                      
a   2018-08-03 00:00:00     30
    2018-08-03 00:05:00    105
b   2018-08-03 00:00:00     35
    2018-08-03 00:05:00    110
c   2018-08-03 00:00:00     40
    2018-08-03 00:05:00    115

### asfreq()

#### asfreq()进行频度转换。

In [15]:
index = pd.date_range('1/1/2000', periods=4, freq='T')
series = pd.Series([0.0, None, 2.0, 3.0], index=index)
df = pd.DataFrame({'s':series})
df

,s
2000-01-01 00:00:00,0.0
2000-01-01 00:01:00,NaN
2000-01-01 00:02:00,2.0
2000-01-01 00:03:00,3.0


#### 将频度转换为30s

In [16]:
df.asfreq(freq='30S')

,s
2000-01-01 00:00:00,0.0
2000-01-01 00:00:30,NaN
2000-01-01 00:01:00,NaN
2000-01-01 00:01:30,NaN
2000-01-01 00:02:00,2.0
2000-01-01 00:02:30,NaN
2000-01-01 00:03:00,3.0


### 将频度转换为2min，不会进行重采样（与resample的不同之处）

In [17]:
df.asfreq(freq='2min')

,s
2000-01-01 00:00:00,0.0
2000-01-01 00:02:00,2.0


### 使用bfill()进行填充

In [18]:
df.asfreq(freq='30S').bfill()

,s
2000-01-01 00:00:00,0.0
2000-01-01 00:00:30,2.0
2000-01-01 00:01:00,2.0
2000-01-01 00:01:30,2.0
2000-01-01 00:02:00,2.0
2000-01-01 00:02:30,3.0
2000-01-01 00:03:00,3.0
